In [31]:
import numpy as np
import time
from matplotlib import pyplot as plt
import NEON_AOP_waveform_tools # written by Tristan, lots of functions
import pandas as pd

In [2]:
# Define the PLS file of the pulsewaves data
PlsFile = r"2015032618_Q780_150326_194954_1_originalpoints.pls"
# there are 2 million pulse points
# You have to have the .wvs available as well.

In [3]:
# Figure out how many pulses there are
fin_pls_file = open(PlsFile,"rb")
fin_pls_file.seek(184,0)
number_of_pulses = np.fromfile(fin_pls_file,np.int64,1)
number_of_pulses

array([24063095])

In [6]:
# Define the pulses we want 
pulses = np.arange(0,10)

In [41]:
xyz_coords = []

In [48]:
for pulse in pulses:
    pulse=[pulse]
    # Read in pulse number info
    # 1) coordinate info
    # 2) received wave
    # 3) outgoing wave
    coord_info, wave_in, wave_out = NEON_AOP_waveform_tools.waveform_read(pulse,PlsFile)
    # print coordinates
    #print(coord_info) # first number is GPS time, next 3 describe the direction of the (3d coordinates), 
    # last three number are the actual coordinates the start of the pulse (x,y,z)
    
    initial_params_out = NEON_AOP_waveform_tools.get_params(wave_out[0])
    initial_params_in = NEON_AOP_waveform_tools.get_params(wave_in[0])
    
    num_peaks_in = np.array(initial_params_in[0,-1]) # number of peaks is saved in this location

    WFtime = np.arange(1,244) # set up time (ns bins)
    
    fit_wave_final = NEON_AOP_waveform_tools.get_fit_wave(initial_params_in, 244, num_peaks_in)
    
    FWHM = (2.355*(initial_params_in[0,2:int(num_peaks_in*3):3]/2**0.5))/2 # full width half max. Grab 3rd, 6th, 9th parameters
    
    trig_loc = initial_params_in[0,1:int(num_peaks_in*3):3] - FWHM
    trig_loc  = np.reshape(trig_loc,(int(num_peaks_in),1))
    
    # normalize trigger locations by the location of the outgoing pulse
    trig_loc = initial_params_in[0,1:int(num_peaks_in*3):3]-FWHM-initial_params_out[0,1]
    trig_loc = np.reshape(trig_loc,(int(num_peaks_in),1))
    
    pulse_coords = (np.matlib.repmat(trig_loc[:,0],3,1)).transpose()*coord_info[:,1:4]+coord_info[:,4:7]
    #print(pulse_coords)
    xyz_coords.append(pulse_coords)
    

read 0 pulses in file in 7.80000000020209e-05 seconds, we are [0.]% done


In [57]:
pd.DataFrame(xyz_coords)

,0
0,"[[531524.6264803635, 4317845.54146926, 2260.42..."
1,"[[531525.0017324772, 4317845.541419738, 2260.3..."
2,"[[531525.9821181245, 4317845.059754713, 2262.0..."
3,"[[531524.2724422665, 4317844.550498687, 2260.2..."
4,"[[531524.6890749062, 4317844.541271165, 2260.2..."
5,"[[531525.21320904, 4317844.605302135, 2260.232..."
6,"[[531525.5660766668, 4317844.568607119, 2260.1..."
7,"[[531524.8767054746, 4317843.541046058, 2260.1..."
8,"[[531525.3144620832, 4317843.540996535, 2260.0..."
9,"[[531525.7538017294, 4317843.568232012, 2260.0..."


AttributeError: 'list' object has no attribute 'shape'